In [ ]:
import os
from collections import defaultdict
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import torch
from dysts.metrics import mae, mse, rmse, smape
from sklearn.decomposition import PCA
from tqdm import tqdm
from tqdm import tqdm

from dystformer.chronos.pipeline import ChronosPipeline
from dystformer.patchtst.pipeline import PatchTSTPipeline
from dystformer.utils import safe_standardize

In [ ]:
if os.path.exists("../custom_style.mplstyle"):
    plt.style.use(["ggplot", "../custom_style.mplstyle"])

In [ ]:
WORK = os.environ.get("WORK", "")
base_dir = f"{WORK}/physics-datasets"
re = 450
fpath = (
    f"{base_dir}/von_karman_street/vortex_street_velocities_Re_{re}_4800timepoints.npz"
)

In [ ]:
vfield = np.load(fpath, allow_pickle=True)
vort_field = (
    np.diff(vfield, axis=1)[..., :-1, 1] + np.diff(vfield, axis=2)[:, :-1, :, 0]
)
vort_field_flattened = vort_field.reshape(vort_field.shape[0], -1)

## Load Model Checkpoints

In [ ]:
run_name = "pft_chattn_emb_w_poly-0"
# run_name = "pft_chattn_noembed_pretrained_correct-0"  # chattn + mlm
# run_name = "pft_linattnpolyemb_from_scratch-0"

pft_model = PatchTSTPipeline.from_pretrained(
    mode="predict",
    pretrain_path=f"/stor/work/AMDG_Gilpin_Summer2024/checkpoints/{run_name}/checkpoint-final",
    device_map="cuda:2",
)

In [ ]:
chronos_ft = ChronosPipeline.from_pretrained(
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_finetune_stand_updated-0/checkpoint-final",
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_mini_ft-0/checkpoint-final",
    "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_bolt_mini-12/checkpoint-final",
    device_map="cuda:3",
    torch_dtype=torch.float32,
)

In [ ]:
chronos_zs = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-mini",
    device_map="cuda:4",
    torch_dtype=torch.float32,
)

## Exploration and Utils

In [ ]:
def plot_model_prediction(
    model,
    data: np.ndarray,
    context_length: int,
    prediction_length: int,
    title: str | None = None,
    show: bool = True,
    transpose: bool = True,
    indices: list[int] | None = None,
    **kwargs,
) -> np.ndarray:
    context = data[:, :context_length]
    groundtruth = data[:, context_length : context_length + prediction_length]
    context_tensor = torch.from_numpy(context.T if transpose else context).float()
    pred = (
        model.predict(context_tensor, prediction_length, **kwargs)
        .squeeze()
        .cpu()
        .numpy()
    )
    if not transpose:
        pred = pred.T

    total_length = context.shape[1] + prediction_length
    context_ts = np.arange(context.shape[1]) / total_length
    pred_ts = np.arange(context.shape[1], total_length) / total_length

    if show:
        if indices is None:
            indices = [0, 1, 2]
        fig = plt.figure(figsize=(15, 4))

        outer_grid = fig.add_gridspec(1, 2, width_ratios=[0.5, 0.5], wspace=0.05)
        gs = outer_grid[1].subgridspec(
            3, 1, height_ratios=[1 / 3] * 3, wspace=0, hspace=0
        )
        ax_3d = fig.add_subplot(outer_grid[0], projection="3d")
        ax_3d.plot(*context[indices], alpha=0.5, color="black", label="Context")
        ax_3d.plot(
            *groundtruth[indices], linestyle="--", color="black", label="Groundtruth"
        )
        ax_3d.plot(*pred.T[indices], color="red", label="Prediction")
        ax_3d.legend(loc="upper right", fontsize=12)
        ax_3d.set_xlabel("$x_{" + str(indices[0]) + "}$")
        ax_3d.set_ylabel("$x_{" + str(indices[1]) + "}$")
        ax_3d.set_zlabel("$x_{" + str(indices[2]) + "}$")
        if title is not None:
            ax_3d.set_title(title)

        axes_1d = [fig.add_subplot(gs[i, 0]) for i in range(3)]
        for i, ax in zip(indices, axes_1d):
            ax.plot(context_ts, context[i], alpha=0.5, color="black")
            ax.plot(pred_ts, groundtruth[i], linestyle="--", color="black")
            ax.plot(pred_ts, pred[:, i], color="red")
            index_str = "{" + str(i) + "}"
            ax.set_ylabel(f"$x_{index_str}$")
            ax.set_aspect("auto")
        axes_1d[-1].set_xlabel("Time")

        plt.show()

    return pred

In [ ]:
n_components = 512
pca = PCA(n_components=n_components)
pca.fit(vort_field_flattened)
X_ts = pca.transform(vort_field_flattened)  # (T, D)
# # safe standardize
# X_ts = safe_standardize(X_ts, axis=0)
eigenvectors = pca.components_  # (D, H*W)

## Show low-rank structure
plt.figure()
plt.plot(np.arange(n_components), pca.explained_variance_ratio_)
plt.semilogy()

## Plot trajectory
plt.figure()
plt.plot(X_ts[:, 0], X_ts[:, 1])

In [ ]:
def reconstruct(
    pca_coeffs: np.ndarray, eigenvectors: np.ndarray, modes: int = -1
) -> np.ndarray:
    if modes == -1:
        modes = pca_coeffs.shape[1]
    return pca_coeffs[:, :modes] @ eigenvectors[:modes, :]

In [ ]:
vort_recon = reconstruct(X_ts, eigenvectors)
vort_recon = vort_recon.reshape(
    vort_field.shape[0], vort_field.shape[1], vort_field.shape[2]
)
plt.figure()
plt.imshow(vort_recon[100 + 512, :, :].T, cmap="seismic")
plt.colorbar(shrink=0.5);

In [ ]:
start = 2048  # ignore transient
stride = 1
subsampled_pca_coeffs = X_ts[start::stride, :]
stand_subsampled_pca_coeffs = safe_standardize(subsampled_pca_coeffs, axis=0)

predictions = plot_model_prediction(
    pft_model,
    stand_subsampled_pca_coeffs.T,
    context_length=512,
    prediction_length=128,
    limit_prediction_length=False,
    sliding_context=True,
    title="Von Karman Vortex Sheet PCA modes",
    indices=[0, 1, 2],
)

## More Utils

In [ ]:
def forecast(
    model,
    data,
    context_length: int,
    prediction_length: int,
    transpose: bool = False,
    standardize: bool = False,
    batch_size: int | None = None,
    time_indices: list[int] | None = None,
    save_path: str | None = None,
    camera_ready: bool = False,
    cmap_name="seismic",
    base_figsize=(5, 5),
    suptitle: str | None = None,
    suptitle_y: float = 1.0,
    **kwargs,
):
    context = data[:context_length, :num_modes]
    groundtruth = data[context_length : context_length + prediction_length, :num_modes]

    if standardize:
        context = safe_standardize(context, axis=0)
        groundtruth = safe_standardize(groundtruth, axis=0)

    torch.cuda.empty_cache()
    context_tensor = torch.from_numpy(context).float()

    if transpose:
        context_tensor = context_tensor.T

    if batch_size is None:
        pred = model.predict(context_tensor, prediction_length, **kwargs)
    else:
        pred = []
        for i in range(0, context_tensor.shape[0], batch_size):
            pred.append(
                model.predict(
                    context_tensor[i : i + batch_size], prediction_length, **kwargs
                )
            )
        pred = torch.cat(pred, dim=0)

    pred = pred.squeeze().detach().cpu().numpy()
    if transpose:
        pred = pred.T

    if standardize:
        pred = safe_standardize(
            pred, axis=0, context=data[start : start + context_length], denormalize=True
        )

    return pred


def plot_predicted_flow(
    prediction: np.ndarray,
    data: np.ndarray,
    eigenvectors: np.ndarray,
    context_length: int,
    prediction_length: int,
    num_modes: int,
    shape: tuple[int, int] = (vort_field.shape[1], vort_field.shape[2]),
    time_indices: list[int] | None = None,
    save_path: str | None = None,
    camera_ready: bool = False,
):
    groundtruth = data[context_length : context_length + prediction_length]

    recon = reconstruct(prediction, eigenvectors, modes=num_modes)
    recon = recon.reshape(prediction_length, shape[0], shape[1])
    groundtruth = reconstruct(groundtruth, eigenvectors, modes=num_modes)
    groundtruth = groundtruth.reshape(prediction_length, shape[0], shape[1])
    vabs = max(groundtruth.min(), groundtruth.max())

    if time_indices is None:
        time_indices = list(range(0, prediction_length, stride))

    aspect_ratio = shape[0] / shape[1]
    fig = plt.figure(
        figsize=(
            base_figsize[0] * (len(time_indices)) / aspect_ratio,
            base_figsize[1] * aspect_ratio,
        )
    )
    gs = fig.add_gridspec(
        2,
        len(time_indices),
        width_ratios=[1] * (len(time_indices)),
        height_ratios=[1, 1],
        wspace=0,
        hspace=0,
    )
    axes = np.array(
        [
            [fig.add_subplot(gs[i, j]) for j in range(len(time_indices))]
            for i in range(2)
        ]
    )
    # for i, index in enumerate([0] + time_indices):
    print(time_indices)
    for i, index in enumerate(time_indices):
        groundtruth_slice = groundtruth[index, :, :]
        recon_slice = recon[index, :, :]
        rax = axes[0, i].imshow(
            recon_slice,
            vmin=-vabs,
            vmax=vabs,
            cmap=cmap_name,
        )
        axes[0, i].spines["top"].set_visible(True)
        axes[0, i].spines["right"].set_visible(True)
        axes[0, i].spines["bottom"].set_visible(True)
        axes[0, i].spines["left"].set_visible(True)
        axes[0, i].spines["top"].set_color("black")
        axes[0, i].spines["right"].set_color("black")
        axes[0, i].spines["bottom"].set_color("black")
        axes[0, i].spines["left"].set_color("black")

        axes[0, i].set_title(
            # f"t={context_length}" + (f" + {index}" if index > 0 else ""),
            "t = context" + (f" + {index + 1}" if index > 0 else ""),
            fontsize=10,
            fontweight="bold",
        )
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])

        gax = axes[1, i].imshow(
            groundtruth_slice,
            vmin=-vabs,
            vmax=vabs,
            cmap=cmap_name,
        )
        axes[1, i].spines["top"].set_visible(True)
        axes[1, i].spines["right"].set_visible(True)
        axes[1, i].spines["bottom"].set_visible(True)
        axes[1, i].spines["left"].set_visible(True)
        axes[1, i].spines["top"].set_color("black")
        axes[1, i].spines["right"].set_color("black")
        axes[1, i].spines["bottom"].set_color("black")
        axes[1, i].spines["left"].set_color("black")

        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])

        circle = plt.Circle(
            (0.5 * shape[1] + 1, 0.145 * shape[0]), 5, fill=True, color="black"
        )
        axes[0, i].add_patch(circle)
        circle = plt.Circle(
            (0.5 * shape[1] + 1, 0.145 * shape[0]), 5, fill=True, color="black"
        )
        axes[1, i].add_patch(circle)

    if not camera_ready:
        axes[0, 0].set_ylabel("Prediction", fontweight="bold", fontsize=16)
        axes[1, 0].set_ylabel("Ground Truth (Low Rank)", fontweight="bold", fontsize=16)

    if suptitle is not None:
        plt.suptitle(
            suptitle, fontweight="bold", fontsize=18, ha="center", y=suptitle_y
        )

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")
    else:
        plt.show()

In [ ]:
def compute_rollout_metrics(
    model,
    data,
    eigenvectors,
    num_modes: int,
    context_length: int,
    prediction_length: int,
    time_interval: int = 64,
    shape: tuple[int, int] = (vort_field.shape[1], vort_field.shape[2]),
    transpose: bool = False,
    batch_size: int | None = None,
    compute_metrics_on_recons: bool = True,
    verbose: bool = False,
    **kwargs,
):
    context = data[:context_length, :num_modes]
    groundtruth = data[context_length : context_length + prediction_length, :num_modes]

    torch.cuda.empty_cache()
    context_tensor = torch.from_numpy(context).float()
    if transpose:
        context_tensor = context_tensor.T

    if batch_size is None:
        pred = model.predict(context_tensor, prediction_length, **kwargs)
    else:
        pred = []
        for i in range(0, context_tensor.shape[0], batch_size):
            pred.append(
                model.predict(
                    context_tensor[i : i + batch_size], prediction_length, **kwargs
                )
            )
        pred = torch.cat(pred, dim=0)

    pred = pred.squeeze().detach().cpu().numpy()
    if transpose:
        pred = pred.T

    metrics = {}

    rollout_time_intervals = np.arange(
        time_interval, prediction_length + time_interval, time_interval
    )
    if verbose:
        print(f"compute_metrics_on_recons: {compute_metrics_on_recons}")
        print(f"rollout time intervals: {rollout_time_intervals}")

    if compute_metrics_on_recons:
        recon = reconstruct(pred, eigenvectors, modes=num_modes)
        recon = recon.reshape(prediction_length, shape[0], shape[1])
        groundtruth = reconstruct(groundtruth, eigenvectors, modes=num_modes)
        groundtruth = groundtruth.reshape(prediction_length, shape[0], shape[1])

        if verbose:
            print(f"recon shape: {recon.shape}")
            print(f"groundtruth shape: {groundtruth.shape}")

        for t in rollout_time_intervals:
            t_slice = slice(0, t)
            metrics[t] = {
                "smape": smape(recon[t_slice], groundtruth[t_slice]) / 2,
                "mse": mse(recon[t_slice], groundtruth[t_slice]),
                "mae": mae(recon[t_slice], groundtruth[t_slice]),
                "rmse": rmse(recon[t_slice], groundtruth[t_slice]),
            }
    else:
        if verbose:
            print(f"pred shape: {pred.shape}")
            print(f"groundtruth shape: {groundtruth.shape}")

        for t in rollout_time_intervals:
            t_slice = slice(0, t)
            metrics[t] = {
                "smape": smape(pred[t_slice], groundtruth[t_slice]) / 2,
                "mse": mse(pred[t_slice], groundtruth[t_slice]),
                "mae": mae(pred[t_slice], groundtruth[t_slice]),
                "rmse": rmse(pred[t_slice], groundtruth[t_slice]),
            }

    return metrics

## Forecasts

In [ ]:
stride = 1
start = 100  # ignore transient
num_modes = 64
context_length = 512
prediction_length = 128
time_indices = [15, 31, 63, 127]

In [ ]:
X_ts.shape

In [ ]:
plot_predicted_flow(
    pft_model,
    X_ts[start::stride],
    context_length=context_length,
    prediction_length=prediction_length,
    sliding_context=True,
    limit_prediction_length=False,
)

plot_predicted_flow(
    pft_predictions,
    X_ts[start::stride],
    eigenvectors,
    context_length=context_length,
    prediction_length=prediction_length,
    num_modes=num_modes,
    time_indices=time_indices,
    camera_ready=True,
)


In [ ]:
chronos_predictions = forecast(
    chronos_ft,
    X_ts[start::stride],
    context_length=context_length,
    prediction_length=prediction_length,
    transpose=True,
    deterministic=True,
    num_samples=1,
    limit_prediction_length=False,
    batch_size=100,
)

plot_predicted_flow(
    chronos_predictions,
    X_ts[start::stride],
    eigenvectors,
    context_length=context_length,
    prediction_length=prediction_length,
    num_modes=num_modes,
    time_indices=time_indices,
    cmap_name="RdBu",
    camera_ready=False,
    base_figsize=(4, 4),
    save_path="figs/von_karman_our_model.pdf",
    suptitle="von Kármán Vortex Street",
    suptitle_y=0.98,
)

In [ ]:
plot_predicted_flow(
    chronos_ft,
    X_ts[start::stride],
    eigenvectors,
    context_length=context_length,
    prediction_length=prediction_length,
    num_modes=num_modes,
    batch_size=100,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    time_indices=time_indices,
    cmap_name="RdBu",
    camera_ready=False,
    save_path="figs/von_karman_chronos_ft.pdf",
    suptitle="Chronos Finetune",
    deterministic=True,
)

In [ ]:
plot_predicted_flow(
    chronos_zs,
    X_ts[start::stride],
    context_length=context_length,
    prediction_length=prediction_length,
    step=64,
    num_windows=10,
    num_samples=1,
    limit_prediction_length=False,
    num_samples=1,
    time_indices=time_indices,
    cmap_name="RdBu",
    camera_ready=False,
    save_path="figs/von_karman_chronos_zs.pdf",
    suptitle="Chronos ZS",
    deterministic=True,
)

## Metrics

In [ ]:
compute_metrics_on_recons = True
use_chronos_deterministic = True

In [ ]:
metrics = compute_rollout_metrics(
    pft_model,
    X_ts[start::stride],
    eigenvectors,
    num_modes=num_modes,
    context_length=context_length,
    prediction_length=512,
    time_interval=64,
    sliding_context=True,
    limit_prediction_length=False,
    compute_metrics_on_recons=compute_metrics_on_recons,
)
smape_metrics = {t: metrics[t]["smape"] for t in metrics.keys()}

In [ ]:
metrics_chronos_ft = compute_rollout_metrics(
    chronos_ft,
    X_ts[start::stride],
    eigenvectors,
    num_modes=num_modes,
    context_length=context_length,
    prediction_length=512,
    time_interval=64,
    batch_size=100,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    compute_metrics_on_recons=compute_metrics_on_recons,
    deterministic=use_chronos_deterministic,
)
smape_metrics_chronos_ft = {
    t: metrics_chronos_ft[t]["smape"] for t in metrics_chronos_ft.keys()
}

In [ ]:
metrics_chronos_zs = compute_rollout_metrics(
    chronos_zs,
    X_ts[start::stride],
    eigenvectors,
    num_modes=num_modes,
    context_length=context_length,
    prediction_length=512,
    time_interval=64,
    batch_size=100,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    compute_metrics_on_recons=compute_metrics_on_recons,
    deterministic=use_chronos_deterministic,
)
smape_metrics_chronos_zs = {
    t: metrics_chronos_zs[t]["smape"] for t in metrics_chronos_zs.keys()
}

In [ ]:
timepoints = list(smape_metrics.keys())
# Plot the SMAPE metrics
plt.figure(figsize=(4, 3))
plt.plot(timepoints, list(smape_metrics.values()), marker="o", label="Our Model")
plt.plot(
    timepoints,
    list(smape_metrics_chronos_ft.values()),
    marker="s",
    label="Chronos 20M Finetune",
)
plt.plot(
    timepoints,
    list(smape_metrics_chronos_zs.values()),
    marker="v",
    label="Chronos 20M",
)
plt.xlabel("Prediction Length", fontweight="bold")
plt.xticks(timepoints)
# plt.title("sMAPE", fontweight="bold")
plt.title("von Kármán Vortex Street", fontweight="bold")
plt.ylabel("sMAPE", fontweight="bold")
plt.legend(loc="lower right", frameon=True)
plt.tight_layout()
plt.savefig("figs/von_karman_smape.pdf", bbox_inches="tight")
plt.show()

In [ ]:
X_ts.shape

In [ ]:
metrics_within_horizon = compute_rollout_metrics(
    pft_model,
    X_ts[start::stride],
    eigenvectors,
    num_modes=num_modes,
    context_length=context_length,
    prediction_length=128,
    time_interval=8,
    sliding_context=True,
    limit_prediction_length=False,
)
smape_within_horizon = {
    t: metrics_within_horizon[t]["smape"] for t in metrics_within_horizon.keys()
}

In [ ]:
timepoints = list(smape_within_horizon.keys())
# Plot the SMAPE metrics
plt.figure(figsize=(4, 3))
plt.plot(timepoints, list(smape_within_horizon.values()), marker="o", label="Our Model")
plt.xlabel("Prediction Length", fontweight="bold")
# plt.xticks(timepoints)
# plt.title("sMAPE", fontweight="bold")
plt.title("von Kármán Vortex Street", fontweight="bold")
plt.ylabel("sMAPE", fontweight="bold")
plt.legend(loc="lower right", frameon=True)
plt.tight_layout()
plt.savefig("figs/von_karman_smape_within_horizon.pdf", bbox_inches="tight")
plt.show()

In [ ]:
len(np.arange(512, 3072, 128))

In [ ]:
# random uniform sample start_times in interval [512, 3072]
n_runs = 10
# rseed = 1234
# rng = np.random.default_rng(rseed)
# start_times = np.random.choice(start_times, size=n_runs, replace=False)
start_times = np.arange(512, 3072, 256)
start_times = start_times.astype(int)
print(f"{len(start_times)} start_times: {start_times}")

In [ ]:
# dict with keys as times t and values as list of smape values for each run
our_model_rollout_metrics = defaultdict(dict)
for start_time in tqdm(start_times, desc="Running rollouts"):
    metrics = compute_rollout_metrics(
        pft_model,
        X_ts[start_time::stride],
        eigenvectors,
        num_modes=num_modes,
        context_length=context_length,
        prediction_length=512,
        time_interval=64,
        sliding_context=True,
        limit_prediction_length=False,
        verbose=False,
    )
    for t, metric in metrics.items():
        for metric_name, metric_val in metric.items():
            if t not in our_model_rollout_metrics[metric_name]:
                our_model_rollout_metrics[metric_name][t] = []
            our_model_rollout_metrics[metric_name][t].append(metric_val)

In [ ]:
# dict with keys as times t and values as list of smape values for each run
chronos_ft_rollout_metrics = defaultdict(dict)
for start_time in tqdm(start_times, desc="Running rollouts"):
    metrics_chronos_ft = compute_rollout_metrics(
        chronos_ft,
        X_ts[start_time::stride],
        eigenvectors,
        num_modes=num_modes,
        context_length=context_length,
        prediction_length=512,
        time_interval=64,
        batch_size=100,
        transpose=True,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=use_chronos_deterministic,
    )
    for t, metric in metrics_chronos_ft.items():
        for metric_name, metric_val in metric.items():
            if t not in chronos_ft_rollout_metrics[metric_name]:
                chronos_ft_rollout_metrics[metric_name][t] = []
            chronos_ft_rollout_metrics[metric_name][t].append(metric_val)

In [ ]:
# dict with keys as times t and values as list of smape values for each run
chronos_zs_rollout_metrics = defaultdict(dict)
for start_time in tqdm(start_times, desc="Running rollouts"):
    metrics_chronos_ft = compute_rollout_metrics(
        chronos_zs,
        X_ts[start_time::stride],
        eigenvectors,
        num_modes=num_modes,
        context_length=context_length,
        prediction_length=512,
        time_interval=64,
        batch_size=100,
        transpose=True,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=use_chronos_deterministic,
    )
    for t, metric_dict in metrics_chronos_ft.items():
        for metric_name, metric_val in metric_dict.items():
            if t not in chronos_zs_rollout_metrics[metric_name]:
                chronos_zs_rollout_metrics[metric_name][t] = []
            chronos_zs_rollout_metrics[metric_name][t].append(metric_val)

In [ ]:
# Calculate mean, standard deviation, and standard error for each model
def calculate_stats(data_dict):
    mean_vals = {t: np.mean(v) for t, v in data_dict.items()}
    median_vals = {t: np.median(v) for t, v in data_dict.items()}
    std_vals = {t: np.std(v) for t, v in data_dict.items()}
    ste_vals = {t: std_vals[t] / np.sqrt(len(data_dict[t])) for t in data_dict.keys()}
    return mean_vals, median_vals, std_vals, ste_vals


# Helper function to plot a model's results with error bands
def plot_model_results(mean_dict, ste_dict, marker, label):
    x_values = list(mean_dict.keys())
    y_values = list(mean_dict.values())
    y_errors = list(ste_dict.values())

    plt.plot(x_values, y_values, marker=marker, label=label)
    plt.fill_between(
        x_values,
        np.array(y_values) - np.array(y_errors),
        np.array(y_values) + np.array(y_errors),
        alpha=0.1,
    )


metric_name = "mae"
metric_name_title = "MAE"
# Calculate statistics for all models
mean_metric_ours, median_metric_ours, _, ste_metric_ours = calculate_stats(
    our_model_rollout_metrics[metric_name]
)
mean_metric_chronos_ft, median_metric_chronos_ft, _, ste_metric_chronos_ft = (
    calculate_stats(chronos_ft_rollout_metrics[metric_name])
)
mean_metric_chronos_zs, median_metric_chronos_zs, _, ste_metric_chronos_zs = (
    calculate_stats(chronos_zs_rollout_metrics[metric_name])
)

# Create the plot
plt.figure(figsize=(4, 3))

# Plot each model
plot_model_results(mean_metric_ours, ste_metric_ours, "o", label="Our Model")
plot_model_results(
    mean_metric_chronos_ft, ste_metric_chronos_ft, "s", label="Chronos 20M Finetune"
)
plot_model_results(
    mean_metric_chronos_zs, ste_metric_chronos_zs, "v", label="Chronos 20M"
)

# Add labels and show
plt.xlabel("Prediction Length", fontweight="bold")
plt.xticks(list(mean_metric_ours.keys()))
plt.title("von Kármán Vortex Street", fontweight="bold")
plt.ylabel(metric_name_title, fontweight="bold")
# Set y-axis to use scientific notation
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))

plt.legend(loc="lower right", frameon=True, fontsize=7)
plt.tight_layout()
plt.savefig(f"figs/vonkarman_all_models_{metric_name}.pdf", bbox_inches="tight")
plt.show()